# Emergency Datahack, 3 трек <a class="anchor" id="0-bullet">

* [Постановка задачи](#п-bullet)
* [Сдача ответов и метрика оценки качества](#с-bullet)
* [Данные](#д-bullet)
* [Тайминг хакатона](#т-bullet)

## Постановка задачи <a class="anchor" id="п-bullet"></a>
[назад к оглавлению](#0-bullet)

[emergencydatahack.ru/track3](https://emergencydatahack.ru/track3)

#### Предпосылки

Национальный проект «Безопасные и качественные автомобильные дороги» к 2024 году предполагает постепенное снижение количества погибших в ДТП и мест концентрации ДТП.


#### Объекты исследования. 

В качестве федеральных автомобильных дорог рассматриваются М-8 «Холмогоры» и Р-21 «Кола».

#### Задача. 

Cпрогнозировать возникновение аварии и её тяжесть на определенном участке федеральной автомобильной дороги на конкретную дату и время. Данные в формате временных рядов. Аварии происходят чрезвычайно редко. События происходят 1 раз за 1000 часов.


Классы целевой переменной:
- ДТП не произойдет
- ДТП произойдет без пострадавших
- ДТП произойдет с пострадавшими
- _Режим ЧС (объявляется от 5 погибших и/или 10 пострадавших)_ - редкий случай, 28 всего, в тестовой выборке не будет вообще. Но круто, если получится оценить, когда это происходит

#### Примечание. 

Способ реагирования МЧС России на ДТП в том числе зависит от того является ли ДТП чрезвычайной ситуацией. Режим чрезвычайной ситуации объявляется от 5 погибших и/или 10 пострадавших, поэтому важно предсказывать не только факт возникновения ДТП, но и его соответствующую категорию.

В тестовой выборке отсутствует класс "Режим ЧС". Подходы к формированию критериев оценки такого вида события оценивается дополнительными баллами.

### Источники данных

- ВНИИ ГОЧС (МЧС России)
- ГИБДД (МВД России)
- Росавтодор
    - интенсивность движения на участках федеральных автодорог
    - аварийно опасные участки
    - ремонтные работы
- Росгидромет
    - атмосферные явления
    - метеорологические явления
- Теле2
    - количество населения передвигавшегося по сегменту дороги по временному интервалу
    - количество вызовов на экстренные номера
- ГЛОНАСС

### Критерии оценки 

<div class="alert alert-block alert-success">
(Внимание, обновление!)
</div>

**60 баллов** распределяется в соответствии с результатом, который ML-модель показала на соревновании. Будет учитываться не место в турнирной таблице напрямую, а логарифмированное значение ошибки на тестовой выборке относительно величины ошибок других участников. По результатам соревнования на очную защиту перед жюри допускается не более 10 команд.

**40 баллов** распределяется по результатам оценки кода, выложенного в открытый репозиторий, а также выступления на очной защите в соответствии со следующими критериями:

- обоснованность выбранных методов для построения модели. Предлагаемая в качестве решения задачи модель машинного обучения является легко интерпретируемой. Участник может объяснить характер зависимости между входными данными (признаками) и принимаемым моделью решением. 
    - К примеру: определенные погодные условия приводят к повышению вероятности возникновения ДТП и эта зависимость прослеживается в процессе принятия решения моделью (20 баллов);
- оригинальность подхода, в частности, использование дополнительных данных из открытых источников для повышения качества модели (10 баллов);
- качество кода: его чистота и понятность (5 баллов);
- качество очной защиты: последовательность и понятность изложения (5 баллов).

**Ожидаемые результаты**

- Краткая аннотация проекта (описание проекта в 2-4 предложениях);
- ML-модель, обученная на тренировочной выборке (сдается на платформе);
- Код, использованный при подготовке модели 
(ссылка на любой открытый репозиторий с кодом, с помощью которого строилась модель, а также кодом, запускающим модель);
- Презентация результатов решения (pptx/pdf/ipynb/...).

## Сдача ответов <a class="anchor" id="с-bullet"></a>
[назад к оглавлению](#0-bullet)

* Выдается тренировочная выборка и тестовая выборка с пропущенными значениями целевой переменной. 

* По мере обновления модели нужно будет загружать в систему `csv`-файлы с проставленными значения целевой переменной.

* Количество попыток - 12

* На private leaderboard показывается результат работы модели на 50% выборки.

* Для оценки на public leaderboard системой автоматически выбирается лучшая модель из private leaderboard.

In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

directory = '/home/common/emergency_datahack/dtp_data_processed/3_track/sample/'

In [2]:
train = pd.read_csv(directory + 'train.csv')
train.head()

,datetime,road_id,road_km,man_injured_num,man_dead_num,car_damaged_num,road_name,lon,lat,data_source,target
0,2018-05-07 16:00:00,9,254,0,0,1.0,"М-8 (E115) =Холмогоры= Москва-Архангельск через Ярославль, Вологду",39.803295,57.545808,gibdd,1
1,2019-02-16 15:00:00,9,216,0,0,3.0,"М-8 (E115) =Холмогоры= Москва-Архангельск через Ярославль, Вологду",39.499154,57.269090,gibdd,1
2,2015-03-22 06:00:00,9,25,1,0,NaN,"М-8 (E115) =Холмогоры= Москва-Архангельск через Ярославль, Вологду",NaN,NaN,gibdd,2
3,2015-01-31 08:00:00,9,28,0,0,NaN,"М-8 (E115) =Холмогоры= Москва-Архангельск через Ярославль, Вологду",NaN,NaN,gibdd,1
4,2020-08-09 00:00:00,9,474,0,0,1.0,"М-8 (E115) =Холмогоры= Москва-Архангельск через Ярославль, Вологду",39.929295,59.292189,gibdd,1


**Ключ: (datetime, road_id, road_km)**

**Сроки: январь 2012 - апрель 2021**

### Метрика

![Cat](macrof1.png)

* Каждое наблюдение отражает ситуацию на конкретное время и на конкретном участке дороги
* Неаварийных наблюдений >> аварий
* Много классов, для которых F1 не определена
* Cохраняем свойство F1 хорошо оценивать предсказание редких событий

<div class="alert alert-block alert-success">(Важно!)</div>

### До отправки ответа использовать проверочный скрипт

**Ограничение платформы**: загрузка файлов на сервер не более 3 Мб при полном файле с ответами около 350 Мб

Как мы помним, ответы сильно смещены в сторону одного класса (ДТП не произошло).

Используем скрипт **check_submit.py**:
- Проверят колонки файла с ответами на соответствие типам и диапазонам значений
- Убирает самый частый класс
- **Проверяет, что файл меньше 3 Мб**

Сервер будет считать все строки с пропущенными значениями id отнесёнными к самому частому классу (ДТП не произошло).


## Данные<a class="anchor" id="д-bullet"></a>
[назад к оглавлению](#0-bullet)

Подробные описания всех датасетов содержатся в кодбуках.

### Росавтодор

#### Информация об интенсивности движения: количество полос, их загруженность, средняя скорость, направление наблюдения

Сроки: январь 2016 - апрель 2020

In [3]:
traffic = pd.read_csv(directory + 'traffic.csv')
traffic.head()

,datetime,road_id,road_km,name,data_id,station_id,lane,lane_count,direction,volume,occupancy,speed,latitude,longitude
0,2016-01-01 00:59:59,9,527,км 526+700,9505403,41108,1,1,backward,32.0,0.80,68.00,59.638016,40.561815
1,2016-01-01 00:59:59,9,181,км 181+000,9898469,40721,1,1,backward,39.0,1.00,73.33,57.001636,39.247588
2,2016-01-01 00:59:59,9,327,км 327+300,9505642,41123,2,2,forward,72.0,1.49,86.00,58.139795,40.141995
3,2016-01-01 00:59:59,9,327,км 327+300,9505642,41123,1,2,forward,4.0,8.00,42.00,58.139795,40.141995
4,2016-01-01 00:59:59,9,327,км 327+300,9505642,41123,1,2,backward,92.0,1.92,75.30,58.139795,40.141995


#### Информация об аварийно опасных участках трасс: диапазон, принадлежность населённому пункту, стабильности участка концентрации ДТП, планируемые работы 

Сроки: январь 2015 - январь 2020

In [4]:
crash_parts = pd.read_csv(directory + 'crash_parts.csv')
crash_parts.head()

,datetime,road_id,road_km,avuch_start,avuch_end,length,avuch_loc,stabchar_type,planactiv_type,planactiv_descr,planactiv_year
0,2017-01-01,9,124,124.77,125.74,970,2,3,4,Дублирование дорожных знаков 3.20 «Обгон запрещен» на щитах с флуорисцентной пленкой желто-зеленого цвета. Щит аварийно-опасный участок.,2018-07-01
1,2017-01-01,9,125,124.77,125.74,970,2,3,4,Дублирование дорожных знаков 3.20 «Обгон запрещен» на щитах с флуорисцентной пленкой желто-зеленого цвета. Щит аварийно-опасный участок.,2018-07-01
2,2017-01-01,9,131,131.88,132.77,890,2,3,4,"Щит аварийно-опасный участок. В 2018 году будет проводиться мониторинг состояния аварийности, разработка и реализация дополнительных мероприятий по БДД, будет выполнена по мере необходимости;",2018-03-01
3,2017-01-01,9,132,131.88,132.77,890,2,3,4,"Щит аварийно-опасный участок. В 2018 году будет проводиться мониторинг состояния аварийности, разработка и реализация дополнительных мероприятий по БДД, будет выполнена по мере необходимости;",2018-03-01
4,2017-01-01,9,145,145.10,145.80,700,2,1,4,"Установлен щит «Аварийно-опасный участок». Изменение режима работы светофорного объекта, проработать вопрос устройства искусственного освещения на перекрестке.",2018-09-01


#### Информация о ремонтных работах на участках трасс

Сроки: январь 2015 - январь 2020

In [5]:
repair = pd.read_csv(directory + 'repair.csv')
repair.head()

,datetime,road_id,road_km,rem_id,year_span,remuch_start,remuch_end,length
0,2016-01-01,9,1213,121312262016,2016,1213.0,1226.218,12.863
1,2016-01-01,9,1214,121312262016,2016,1213.0,1226.218,12.863
2,2016-01-01,9,1215,121312262016,2016,1213.0,1226.218,12.863
3,2016-01-01,9,1216,121312262016,2016,1213.0,1226.218,12.863
4,2016-01-01,9,1217,121312262016,2016,1213.0,1226.218,12.863


### Росгидромет

Сроки: январь 2015 - декабрь 2020

#### Атмосферные явления: вид, интенсивность, время начала и конца

In [6]:
atmos = pd.read_csv(directory + 'atmos.csv')
atmos.sample(3)

,road_id,station,lat,lon,phenomenon,phenomenon_q,intensity,intensity_q,start_date,start_ts,start_q,end_date,end_ts,end_q
12,9,ALEKSAN,56.4,38.75,дождь,Значение элемента достоверно,слабая интенсивность,Значение элемента достоверно,2015-01-02,12:00,Значение элемента достоверно,2015-01-02,13:00,Значение элемента достоверно
10,9,ALEKSAN,56.4,38.75,дождь,Значение элемента достоверно,умеренная интенсивность,Значение элемента достоверно,2015-01-02,8:20,Значение элемента достоверно,2015-01-02,9:00,Значение элемента достоверно
19,9,ALEKSAN,56.4,38.75,снег ливневый мокрый,Значение элемента достоверно,умеренная интенсивность,Значение элемента достоверно,2015-01-03,6:40,Значение элемента достоверно,2015-01-03,7:00,Значение элемента достоверно


#### Метеорологические явления: время измерения, явления и их критерии качества

In [7]:
meteo = pd.read_csv(directory + 'meteo.csv')
meteo.sample(3)

,road_id,station,lat,lon,measure_dt,vsp_1,vsp_1_q,vsp_2,vsp_2_q,vsp_3,vsp_3_q,visib,visib_q,clouds,clouds_q,weather_range,weather_range_q,weather_on_measure,weather_on_measure_q,wind_dir,wind_dir_q,avg_wind,avg_wind_q,max_wind,max_wind_q,precip,precip_q,temp_on_measure,temp_on_measure_q,temp_min,temp_min_q,temp_max,temp_max_q,humidity,humidity_q,pressure,pressure_q
2,9,ALEKSAN,56.4,38.75,2015-01-01 00:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,97.0,Значение элемента достоверно,9.0,Значение элемента достоверно,Пасмурно или облачность более 5 баллов,Значение элемента достоверно,Облака развиваются,Значение элемента достоверно,261.0,Значение элемента достоверно,1.0,Значение элемента достоверно,3.0,Значение элемента достоверно,NaN,Значение элемента отсутствует,-4.8,Значение элемента достоверно,-7.0,Значение элемента достоверно,-4.8,Значение элемента достоверно,97.0,Значение элемента достоверно,984.8,Значение элемента достоверно
6,9,ALEKSAN,56.4,38.75,2015-01-01 12:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,97.0,Значение элемента достоверно,10.0,Значение элемента достоверно,Снег или дождь со снегом,Значение элемента достоверно,Облака развиваются,Значение элемента достоверно,243.0,Значение элемента достоверно,2.0,Значение элемента достоверно,5.0,Значение элемента достоверно,NaN,Значение элемента отсутствует,-0.8,Значение элемента достоверно,-1.5,Значение элемента достоверно,-0.7,Значение элемента достоверно,94.0,Значение элемента достоверно,985.9,Значение элемента достоверно
17,9,ALEKSAN,56.4,38.75,2015-01-03 21:00:00,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,16.0,Значение элемента достоверно,93.0,Значение элемента достоверно,10.0,Значение элемента достоверно,Ливневые осадки,Значение элемента достоверно,Ливневый снег умеренный или сильный,Значение элемента достоверно,196.0,Значение элемента достоверно,4.0,Значение элемента достоверно,9.0,Значение элемента достоверно,NaN,Значение элемента отсутствует,0.1,Значение элемента достоверно,0.0,Значение элемента достоверно,0.7,Значение элемента достоверно,96.0,Значение элемента достоверно,964.3,Значение элемента достоверно


### Tele2: количество населения на участках дороги и вызовов на экстренные номера

In [8]:
tele2 = pd.read_csv(directory + 'tele2_data.csv')
tele2.head()

,datetime,road_id,road_km,cnt_subs
0,2020-12-01 00:00:00,9,0,85
1,2020-12-01 01:00:00,9,0,81
2,2020-12-01 02:00:00,9,0,69
3,2020-12-01 03:00:00,9,0,55
4,2020-12-01 04:00:00,9,0,69


### Данные геосервисов ГЛОНАСС и Яндекса.Геокодер: отображение километра дороги в координаты

In [9]:
geo_data = pd.read_csv(directory + 'geo_data.csv')
geo_data.head()

,road_id,road_km,km_name,lat_geoc,lon_geoc,lat_glonass,lon_glonass
0,9,0,"М-8 Холмогоры, 0-й километр",55.755975,37.613846,56.218809,38.087886
1,9,1,"М-8 Холмогоры, 1-й километр",55.758915,37.620349,56.554579,38.587849
2,9,2,"М-8 Холмогоры, 2-й километр",55.761990,37.633704,56.546578,38.594389
3,9,3,"М-8 Холмогоры, 3-й километр",55.768684,37.641095,NaN,NaN
4,9,4,"М-8 Холмогоры, 4-й километр",55.768529,37.631879,NaN,NaN


<div class="alert alert-block alert-success">
(Важно!)
</div>

### По километру дороги получить ближайшую станцию: **match_station.ipynb**

## Тайминг хакатона<a class="anchor" id="т-bullet"></a>
[назад к оглавлению](#0-bullet)

28 мая  
18:00 - Открытие хакатона  
18:20 - Выдача презентаций с задачами и данных
19:30 - Q&A сессия с экспертами по задачам  

29 мая  
11:00 - Чек-поинт #1  
15:00 - Мастер-класс от компании Tele2 "Геоаналитика на транспорте"  
18:00 - Чек-поинт #2  

30 мая  
11:00 - Дедлайн загрузки решений  
11:00 - 15:00 - Проверка кода  
15:00 - 17:00 - Защита проектов  
17:00 - 18:00 - Подведение итогов  
18:00 - Объявление победителей, завершение хакатона